## CATEGORICAL DATA

### BACKGROUND AND MOTIVATION

In [1]:
# unique/value_counts - extract the distinct values from an array and commute frequencies
import numpy as np
import pandas as pd

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [5]:
# dimension tables - contains distinct values
# stores the primary obs as integer keys referencing the dimension table

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])

In [7]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [8]:
dim

0     apple
1    orange
dtype: object

In [9]:
# use 'take' method to restore the original Series of strings
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [10]:
# this representation as integers is called the 'categorical' representation
# a.k.a dictionary-encoded
# array of distinct values can be called as categories/dictionary, or levels od the data
# integer values that refer the categories are called category codes

In [11]:
# categorical representation allows performance improvements for data analytics
# can perform transformations on categories without modifying codes

### CATEGORICAL TYPE IN PANDAS

In [12]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

In [18]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,4,3.659376
1,1,orange,5,1.453703
2,2,apple,6,0.126725
3,3,apple,11,3.386531
4,4,apple,10,3.345139
5,5,orange,4,3.976485
6,6,apple,9,1.574539
7,7,apple,3,3.062627


In [20]:
# df['fruit'] - array of Python string objs
# convert it to categorical
# values of fruit_cat are not NumPy array but instance of pandas.Categorical
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [21]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [24]:
# convert a DF column to categorical by assigning the converted result
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [25]:
# create pandas.Categorical from other types of Python sequences
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [26]:
# if categorical encoded data has been fetched from another source
# use from_codes constructor
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

In [27]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [28]:
# without explicit params, categorical conversions assume no specific ordering of the categories
# when using from_codes, one can indicate category having a meaningful ordering
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [29]:
# convert unordered categorical instance to ordered
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [30]:
# categorical array can consist of any immutable value types

### COMPUTATIONS WITH CATEGORICALS

In [31]:
# groupby functions work better with categoricals
# some functions also utilize ordered flag

In [32]:
np.random.seed(12345)

In [33]:
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [34]:
# commute quartile binning of this data and extract some statistics
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [35]:
# quartile names using labels arg
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [36]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [38]:
bins.categories

Index(['Q1', 'Q2', 'Q3', 'Q4'], dtype='object')

In [39]:
# labeled bins categorical do not contain info about the bins edges in the data
# use groupby to extract summary stats
bins = pd.Series(bins, name="quartile")
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()

In [40]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [41]:
# quatile column maintains original categorical info
# including ordering from bins
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### BETTER PERFORMANCE WITH CATEGORICALS

In [42]:
# converting dataset to categorical can show substantial performance gains
# categorical version of DF column will use less memory

In [44]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N//4))

In [46]:
# now convert labels to categorical
categories = labels.astype('category')

In [48]:
labels.memory_usage()

80000080

In [49]:
categories.memory_usage()

10000272

In [50]:
# conversion to category is a one-time cost
%time _ = labels.astype('category')

CPU times: user 393 ms, sys: 40.8 ms, total: 434 ms
Wall time: 432 ms


In [51]:
# GroupBy ops can be faster with categoricals
# underlying algorithms use integer-based codes array instead of an array of strings

### CATEGORICAL METHODS

In [52]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [53]:
# 'cat' provides access to categorical methods
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [54]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [55]:
# if we know that the actual set of categories extend beyond four values
# use set_categories method to change them

In [56]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [57]:
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [61]:
# data remains unchanged
# new categories will be reflected in ops that use them
# value_counts respects the categories
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [62]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64